In [1]:
import sympy

# Quasistatic Poroelasticity

The governing equations are

\begin{gather}
% Solution
\vec{s}^{T} = \left(\vec{u} \quad p \quad \vec{\epsilon}_v \right) \\
% Displacement
\vec{f}(\vec{x},t) + \nabla \cdot \boldsymbol{\sigma}(\vec{u},p) = 0 \text{ in } \Omega \\
% Pressure
\frac{\partial \zeta(\vec{u},p)}{\partial t } - \gamma(\vec{x},t) + \nabla \cdot \vec{q}(p) = 0 \text{ in } \Omega \\
% Neumann traction
\boldsymbol{\sigma} \cdot \vec{n} = \vec{\tau}(\vec{x},t) \text{ on } \Gamma_{\tau} \\
% Dirichlet displacement
\vec{u} = \vec{u}_{0}(\vec{x}, t) \text{ on } \Gamma_{u} \\
% Neumann flow
\vec{q} \cdot \vec{n} = q_{0}(\vec{x}, t) \text{ on } \Gamma_{q} \\
% Dirichlet pressure
p = p_{0}(\vec{x},t) \text{ on } \Gamma_{p}
\end{gather}

where

\begin{gather}
%
  \vec{q}(p) = -\frac{\boldsymbol{k}}{\mu_{f}}(\nabla p - \vec{f}_f), \\
%
  \zeta(\vec{u},p) = \alpha (\nabla \cdot \vec{u}) + \frac{p}{M}, \\
%
  \boldsymbol{\sigma}(\vec{u},p) = \boldsymbol{C}:\boldsymbol{\epsilon} - \alpha p \boldsymbol{I}
  = \lambda \boldsymbol{I} \epsilon_{v} + 2 \mu \boldsymbol{\epsilon}  - \alpha \boldsymbol{I} p, \\
\lambda = K_{d} - \frac{2}{3} \mu, \\
  \frac{1}{M} = \frac{\alpha-\phi}{K_s} + \frac{\phi}{K_f}, \\
  \alpha = 1 - \frac{K_d}{K_f}, \\
\epsilon_{v} = \nabla \cdot \vec{u},
\end{gather}

$M$ denotes the Biot modulus, $\alpha$ denotes the Biot coefficient, $1/M$ is the specific storage coefficient at constant strain, $K_d$ denotes the bulk modulus of the drained system, $K_s$ denotes the bulk modulus of the solid, and $K_f$ denotes the bulk modulus of the fluid, $\mu$ denotes the shear modulus, and $\epsilon_{v}$ denotes the volumetric strain.

## Test Case: Linear Gradient in Fluid Pressure

We consider a linear gradient in fluid pressure along the x direction,

\begin{equation}
p(x) = p_0 \left( 1 - \frac{x}{L} \right).
\end{equation}

Solving for $\vec{q}$ with $\vec{f}_f = \vec{0}$, we have

\begin{aligned}
q_x &= \frac{k}{\mu_f} \frac{p_0}{L}\\
q_y &= 0
\end{aligned}

Solving the elasticity equation leads to

\begin{aligned}
u_x(x) &= -\frac{1}{2} \frac{\alpha p_0}{\lambda + 2\mu} \frac{x^2}{L}, \\
u_y &= 0, \\
\epsilon_v &= - \frac{\alpha p_0}{\lambda + 2\mu} \frac{x}{L}, \\
\sigma_{xx} &= -\alpha p_0, \\
\sigma_{yy} &= -\alpha p_0 \left( 1 + \frac{x}{L} \left( 1 - \frac{\lambda}{\lambda+2\mu}\right)\right), \\
\sigma_{xy} &= 0.
\end{aligned}

### Verify the analytical solution

In [18]:
# Define symbolic variable
x, y = sympy.symbols("x, y")

# Define symbolic constants
p0, L = sympy.symbols("p0, L")

# Material parameters
λ, μ, ϕ, α, μf, k, ρb, ρf, M = sympy.symbols("λ, μ, ϕ, α, μf, k, ρb, ρf, M")

# Analytical solution
ux = -0.5*α*p0 /(λ+2*μ) * x**2 / L
uy = 0 * x
p = p0 * (1 - x/L)

# Derivatives for governing equations
ux_x = ux.diff(x)
ux_y = ux.diff(y)
uy_x = uy.diff(x)
uy_y = uy.diff(y)
p_x = p.diff(x)
p_y = p.diff(y)
grad_p = sympy.Matrix([p_x, p_y])

# Body force for fluid phase (assume 0)
ff = sympy.Matrix([0., 0.])

# Darcy flux; Generalized Dacy's law
q = -(k/μf)*( grad_p + - ff)

# Strain
ϵxy = (ux_y + uy_x) / 2
ϵ = sympy.Matrix([[ux_x, ϵxy],[ϵxy, uy_y]])
ϵv = sympy.trace(ϵ)

# Stress
σ  = λ * ϵv * sympy.eye(2) + 2 * μ * ϵ - α * sympy.eye(2) * p

# Variation of fluid content
ζ = α * ϵv + p/M

# Divergence of stress
div_σ = sympy.Matrix([σ[0,0].diff(x) + σ[0,1].diff(y), σ[1,0].diff(x) + σ[1,1].diff(y)])

# Divergence of flux
div_q = q[0].diff(x) + q[1].diff(y)

In [19]:
sympy.simplify(σ)

Matrix([
[-p0*α,                                         0],
[    0, p0*α*(-L*λ - 2*L*μ + 2*x*μ)/(L*(λ + 2*μ))]])

In [24]:
sympy.simplify(q)

Matrix([
[k*p0/(L*μf)],
[          0]])

In [25]:
sympy.simplify(div_σ)

Matrix([
[0],
[0]])

In [27]:
sympy.simplify(div_q)

0